In [1]:
import pickle
from sklearn.naive_bayes import GaussianNB 
import numpy as np
import operator

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, recall_score, precision_score
from bayes_opt import BayesianOptimization

from general_functions import create_balanced_dataset

In [2]:
file = open("dataset/zone_4.pickle", "rb")
zone_4 = pickle.load(file)

file = open("dataset/zone_7.pickle", "rb")
zone_7 = pickle.load(file)
file = None

In [3]:
with open("dataset/zone_4_resampled.pickle", "rb") as file:
    zone_4_resampled = pickle.load(file)
with open("dataset/zone_7_resampled.pickle", "rb") as file:
    zone_7_resampled = pickle.load(file)

In [4]:
experiment_arr = [(zone_4_resampled, zone_7), (zone_7_resampled, zone_4)]

## Features to use based on feature importances from Random Forests

In [5]:
features_to_use = [
    ['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams',
     'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams',
     'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4',
     'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4',
     'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified',
     'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4',
     'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2',
     'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6',
     'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams',
     'hpmf_filter_no_streams', 'hpmf_mean_2', 'hpmf_mean_3', 'hpmf_mean_4', 'hpmf_mean_6', 'hpmf_median_2',
     'hpmf_median_4', 'hpmf_median_6', 'hpmf_min_2', 'hpmf_min_4', 'hpmf_min_6', 'hpmf_max_2', 'hpmf_max_4',
     'hpmf_max_6', 'hpmf_std_2', 'hpmf_std_4', 'hpmf_std_6', 'slope_non_ditch', 'slope_mean_2', 'slope_mean_3',
     'slope_mean_4', 'slope_mean_6', 'slope_median_2', 'slope_median_4', 'slope_median_6', 'slope_min_2', 'slope_min_4',
     'slope_min_6', 'slope_max_2', 'slope_max_4', 'slope_std_2', 'slope_std_4', 'slope_std_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'hpmf_mean_4', 'hpmf_mean_3',
     'impundment_amplified', 'hpmf_median_4', 'impoundment_amplified_no_streams', 'skyview_max_6', 'skyview_gabor',
     'hpmf_min_2', 'impundment_max_2', 'impundment_std_4', 'skyview_non_ditch', 'skyview_gabor_no_streams', 'slope_min_6',
     'skyview_max_4', 'impundment_median_2', 'impundment_max_6', 'slope_non_ditch', 'impundment_mean_6', 'hpmf_filter',
     'impundment_std_6', 'hpmf_filter_no_streams', 'impundment_median_6', 'hpmf_mean_6', 'slope_min_4', 'slope_std_6',
     'hpmf_mean_2', 'slope_median_6', 'impundment_max_4', 'impundment_std_2', 'hpmf_gabor', 'hpmf_min_4', 'skyview_min_6',
     'skyview_median_6', 'slope_mean_6', 'hpmf_gabor_no_streams', 'skyview_std_6', 'slope_min_2', 'hpmf_median_2',
     'skyview_max_2', 'hpmf_std_6', 'slope_std_4', 'hpmf_min_6', 'slope_max_4', 'skyview_mean_6', 'hpmf_max_6', 'skyview_min_4'],
    ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'impundment_amplified',
     'impoundment_amplified_no_streams', 'hpmf_median_4', 'skyview_max_6', 'hpmf_mean_4', 'skyview_gabor',
     'skyview_non_ditch', 'slope_min_6', 'skyview_max_4', 'skyview_gabor_no_streams', 'slope_median_6', 'impundment_max_6',
     'hpmf_filter', 'skyview_median_6', 'slope_non_ditch', 'impundment_std_4', 'slope_std_6', 'hpmf_filter_no_streams',
     'impundment_median_6', 'hpmf_mean_6', 'slope_min_4', 'hpmf_mean_3', 'impundment_mean_6', 'slope_min_2', 'slope_mean_6',
     'skyview_max_2', 'slope_std_4', 'impundment_median_2', 'hpmf_std_6', 'skyview_min_6', 'impundment_std_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_amplified', 'hpmf_median_4',
     'impoundment_amplified_no_streams', 'impundment_mean_2', 'skyview_max_6', 'skyview_gabor', 'skyview_non_ditch',
     'slope_min_6', 'hpmf_mean_6', 'skyview_median_6', 'skyview_max_4', 'skyview_min_6', 'impundment_max_6', 'slope_std_6',
     'slope_median_6', 'skyview_gabor_no_streams', 'hpmf_std_6', 'impundment_std_4', 'slope_std_4', 'hpmf_filter',
     'impundment_median_6', 'slope_mean_6'],
    ['impundment_mean_3', 'hpmf_median_4', 'impundment_mean_4', 'impundment_amplified', 'skyview_non_ditch', 'skyview_max_6',
     'hpmf_filter', 'impoundment_amplified_no_streams', 'hpmf_mean_6', 'slope_min_6', 'impundment_max_6', 'skyview_median_6',
     'skyview_gabor', 'skyview_min_6', 'skyview_max_4', 'slope_median_6', 'impundment_mean_2', 'impundment_median_6',
     'hpmf_std_6', 'slope_std_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'hpmf_median_4', 'skyview_gabor', 'impundment_amplified', 'skyview_max_6',
     'skyview_non_ditch', 'impundment_max_6', 'hpmf_filter', 'impoundment_amplified_no_streams', 'slope_min_6',
     'hpmf_mean_6', 'slope_median_6', 'slope_std_6', 'skyview_median_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'skyview_max_6', 'hpmf_median_4', 'skyview_gabor', 'impundment_max_6',
     'impundment_amplified', 'hpmf_filter', 'slope_min_6', 'skyview_non_ditch']
]

## Testing Naive Bayes performance for different feature spaces

In [6]:
for most_important_features in features_to_use:
    
    num_features = len(most_important_features)
    print(f"Features used in experiment:\n{most_important_features}")
    print("\n")
    y_test_all = []
    pred_all = []
    
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None
        
        
        clf = GaussianNB()

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        pred = clf.predict(X_test)

        for i, pred_var in enumerate(pred):
            y_test_all.append(y_test[i])
            pred_all.append(pred_var)

    print("\n")
    print(f"Amount of features used:   {num_features}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")

Features used in experiment:
['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams', 'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams', 'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4', 'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4', 'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified', 'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4', 'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2', 'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6', 'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams', 'hpmf_filter_no_st